In [32]:
# import pandas as pd
# df = pd.read_csv("Covid_19.csv")
# print(df.columns)
# df.head()

In [33]:
# Create corrution

In [34]:
import pandas as pd, numpy as np, os

df = pd.read_csv("Covid_19.csv")
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# --- enlarge dataset to about 50 MB ---
target_size_mb = 50
row_size = df.memory_usage(deep=True).sum() / len(df)
target_rows = int((target_size_mb * 1024**2) / row_size)
df_large = pd.concat([df]* (target_rows//len(df)+1), ignore_index=True)
df_large = df_large.sample(n=target_rows, random_state=42).reset_index(drop=True)

# --- strong corruption: 40 % noise, 20 % outliers ---
for col in num_cols:
    mask = np.random.rand(len(df_large)) < 0.4
    df_large.loc[mask, col] += np.random.normal(0, df_large[col].std()*6, mask.sum())
    mask_out = np.random.rand(len(df_large)) < 0.2
    df_large.loc[mask_out, col] = df_large[col].mean()*np.random.uniform(10,30, mask_out.sum())

# small random shuffling to break order
df_large = df_large.sample(frac=1, random_state=1).reset_index(drop=True)

out_path = "Covid_19_corrupted_50MB.csv"
df_large.to_csv(out_path, index=False)
print("Corrupted dataset saved:", out_path,
      f"({os.path.getsize(out_path)/(1024**2):.2f} MB)")


✅ Corrupted dataset saved: Covid_19_corrupted_50MB.csv (38.55 MB)


In [35]:
# New

In [30]:
# Another

In [2]:
# ===============================================
# COVID-19 RL MODEL: BASELINE + CR-PMLE VERSION
# With Positive Robustness Improvements
# ===============================================

import pandas as pd
import numpy as np
import torch, torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import trange

# -----------------------------------------------
# LOAD DATASET
# -----------------------------------------------
print("Loading dataset...")
df = pd.read_csv("Covid_19_corrupted_50MB.csv")  # corrupted dataset

# Select numeric features safely
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

for bad_col in ["date", "Date", "DATE", "year", "month", "day"]:
    if bad_col in num_cols:
        num_cols.remove(bad_col)

if len(num_cols) == 0:
    raise ValueError("No numeric columns found in dataset. Please check your CSV.")

state_cols = num_cols[:8]  # use up to 8 numeric features
print(f"Using columns: {state_cols}")

# -----------------------------------------------
# BUILD TRANSITIONS SAFELY
# -----------------------------------------------
loc_col = None
for c in df.columns:
    if c.lower() in ("location", "country", "country_region", "country_name", "region"):
        loc_col = c
        break

if loc_col is None:
    df["_location"] = "ALL"
    loc_col = "_location"

df_sorted = df.sort_values(by=[loc_col])
for c in state_cols:
    df_sorted["next_" + c] = df_sorted.groupby(loc_col)[c].shift(-1)

df_pairs = df_sorted.dropna(subset=["next_" + c for c in state_cols])
if len(df_pairs) < 10:
    raise ValueError(
        f"Not enough valid rows after creating next-state pairs ({len(df_pairs)} rows found)."
    )

df_pairs = df_pairs.reset_index(drop=True)
print(f"Created {len(df_pairs)} transition pairs for training.")

# -----------------------------------------------
# CLEAN NaN VALUES BEFORE NORMALIZATION
# -----------------------------------------------
data_all = pd.concat(
    [df_pairs[state_cols], df_pairs[["next_" + c for c in state_cols]]], axis=1
)
if data_all.isnull().sum().sum() > 0:
    print(f"Found {data_all.isnull().sum().sum()} NaN values — filling with column means.")
    data_all = data_all.fillna(data_all.mean())

X = data_all[state_cols].values.astype(np.float32)
Y = data_all[["next_" + c for c in state_cols]].values.astype(np.float32)

# -----------------------------------------------
# NORMALIZE AND SPLIT
# -----------------------------------------------
scaler_X = MinMaxScaler()
scaler_Y = MinMaxScaler()
X = scaler_X.fit_transform(X)
Y = scaler_Y.fit_transform(Y)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(f"Train: {X_train.shape}, Test: {X_test.shape}")

# -----------------------------------------------
# DEFINE MODEL
# -----------------------------------------------
class MLP(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, out_dim)
        )
    def forward(self, x): return self.net(x)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# -----------------------------------------------
# PART 1: BASELINE MODEL (MLE)
# -----------------------------------------------
print("\nTraining baseline MLE model...")
model = MLP(X_train.shape[1], Y_train.shape[1]).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

ds = DataLoader(TensorDataset(torch.tensor(X_train), torch.tensor(Y_train)), batch_size=256, shuffle=True)
for epoch in range(15):
    for xb, yb in ds:
        xb, yb = xb.to(DEVICE), yb.to(DEVICE)
        opt.zero_grad()
        loss = loss_fn(model(xb), yb)
        loss.backward()
        opt.step()
print("Baseline model training complete.")

model.eval()
with torch.no_grad():
    preds = model(torch.tensor(X_test).float().to(DEVICE)).cpu().numpy()
mse_baseline = mean_squared_error(Y_test, preds)
print(f"Baseline MSE (before correction): {mse_baseline:.6f}")

torch.save(model.state_dict(), "covid_model_baseline.pth")

# -----------------------------------------------
# PART 2: CR-PMLE (ROBUST MODEL)
# -----------------------------------------------
print("\nStarting CR-PMLE correction (robust training)...")

def train_temp(X, Y, epochs=5):
    m = MLP(X.shape[1], Y.shape[1]).to(DEVICE)
    opt = torch.optim.Adam(m.parameters(), lr=1e-3)
    loss_fn = nn.MSELoss()
    ds = DataLoader(TensorDataset(torch.tensor(X), torch.tensor(Y)), batch_size=256, shuffle=True)
    for _ in range(epochs):
        for xb, yb in ds:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad()
            loss = loss_fn(m(xb), yb)
            loss.backward()
            opt.step()
    return m

# Ensemble predictions for uncertainty
K = 4
preds = []
for k in range(K):
    idx = np.random.choice(len(X_train), len(X_train), replace=True)
    m = train_temp(X_train[idx], Y_train[idx])
    m.eval()
    with torch.no_grad():
        preds.append(m(torch.tensor(X_train).float().to(DEVICE)).cpu().numpy())
preds = np.stack(preds, axis=0)
var = preds.var(axis=0).mean(axis=1)

# ---------------------------------------------
# STRONGER CR-PMLE WEIGHTING (positive robustness)
# ---------------------------------------------
alpha = 5.0  # high sensitivity
var_norm = (var - var.min()) / (var.max() - var.min() + 1e-8)
weights = np.exp(-alpha * var_norm)
weights = np.clip(weights, 1e-4, 1.0)
weights = weights / weights.mean()

# ---------------------------------------------
# TRAIN WEIGHTED MODEL
# ---------------------------------------------
model_w = MLP(X_train.shape[1], Y_train.shape[1]).to(DEVICE)
model_w.load_state_dict(model.state_dict())  # warm-start from baseline
opt = torch.optim.Adam(model_w.parameters(), lr=5e-4)
loss_fn = nn.MSELoss(reduction='none')

Xtr_t = torch.tensor(X_train).float().to(DEVICE)
Ytr_t = torch.tensor(Y_train).float().to(DEVICE)
w_t = torch.tensor(weights).float().to(DEVICE)

for epoch in trange(20, desc="CR-PMLE Training"):
    perm = np.random.permutation(len(X_train))
    for i in range(0, len(perm), 256):
        ids = perm[i:i+256]
        xb, yb, wb = Xtr_t[ids], Ytr_t[ids], w_t[ids]
        opt.zero_grad()
        pred = model_w(xb)
        loss = (loss_fn(pred, yb).mean(1) * wb).mean()
        loss.backward()
        opt.step()

print("CR-PMLE model training complete.")

model_w.eval()
with torch.no_grad():
    preds_w = model_w(torch.tensor(X_test).float().to(DEVICE)).cpu().numpy()
mse_weighted = mean_squared_error(Y_test, preds_w)
print(f"Weighted MSE (after correction): {mse_weighted:.6f}")

torch.save(model_w.state_dict(), "covid_model_weighted.pth")

# -----------------------------------------------
# FINAL COMPARISON
# -----------------------------------------------
print("\n================= RESULTS =================")
print(f"Before Correction (MLE)   MSE: {mse_baseline:.6f}")
print(f"After Correction (CR-PMLE) MSE: {mse_weighted:.6f}")
improvement = ((mse_baseline - mse_weighted) / mse_baseline) * 100
print(f"Improvement in Robustness: {improvement:.2f}%")
print("===========================================")


Loading dataset...
Using columns: ['total_cases', 'new_cases', 'new_cases_smoothed', 'total_deaths', 'new_deaths', 'new_deaths_smoothed', 'total_cases_per_million', 'new_cases_per_million']
Created 58876 transition pairs for training.
Found 9170 NaN values — filling with column means.
Train: (47100, 8), Test: (11776, 8)

Training baseline MLE model...
Baseline model training complete.
Baseline MSE (before correction): 0.005898

Starting CR-PMLE correction (robust training)...


CR-PMLE Training: 100%|████████████████████████████████████████████████████████████████| 20/20 [00:26<00:00,  1.33s/it]

CR-PMLE model training complete.
Weighted MSE (after correction): 0.005781

================= RESULTS =================
Before Correction (MLE)   MSE: 0.005898
After Correction (CR-PMLE) MSE: 0.005781
Improvement in Robustness: 1.98%
